## Analisis inicial

In [14]:
from IPython.display import display
import folium
import pandas as pd

In [15]:
# Cargar los archivos Excel
sensores = pd.read_excel("../sensores_airenuevoleon.xlsx")
registros = pd.read_excel("../registros_airenuevoleon.xlsx")

# Mostrar dimensiones
print("Sensores:", sensores.shape)
print("Registros:", registros.shape)

Sensores: (15, 11)
Registros: (959387, 17)


In [16]:
print("\n Datos de Sensores:")
print("\nColumnas sensores:", sensores.columns.tolist())
print(sensores.head())


 Datos de Sensores:

Columnas sensores: ['Sensor_id', 'Zona', 'Calle', 'Municipio', 'CP', 'Latitud', 'Longitud', 'Sistema', 'Bot_id', 'USE_IN_BOTS', 'website_visible']
  Sensor_id       Zona                                             Calle  \
0      ANL1  Guadalupe                    AV. ELOY CAVAZOS Y PABLO LIVAS   
1     ANL10    Apodaca                                   MARIANO ABASOLO   
2     ANL11  Monterrey                           PREPA TEC GARZA LAGUERA   
3     ANL12   Obispado                                         5 DE MAYO   
4     ANL13     Juarez  LAZARO GARZA AYALA ESQ. CON GRAL. SANTIAGO TAPIA   

   Municipio     CP  Latitud  Longitud        Sistema  Bot_id  USE_IN_BOTS  \
0  Guadalupe  67140    25.67   -100.25  AireNuevoLeon       3            1   
1    Apodaca  66000    25.78   -100.17  AireNuevoLeon       3            1   
2  Monterrey  64989    25.62   -100.27  AireNuevoLeon       3            1   
3  Monterrey  64040    25.68   -100.34  AireNuevoLeon       3 

In [17]:
print("\n Datos de Registros:")
print("Columnas registros:", registros.columns.tolist())
print(registros.head())


 Datos de Registros:
Columnas registros: ['Registros_id', 'Dia', 'PM10', 'PM25', 'O3', 'CO', 'NO1', 'NO2', 'NOx', 'SO2', 'Sensor_id', 'TEMPERATURA', 'LLUVIA', 'PRESIONATM', 'HUMEDAD', 'RS', 'VIENTOVEL']
   Registros_id        Dia   PM10   PM25     O3    CO     NO1     NO2     NOx  \
0       2440510 2025-04-04   65.0  23.52  0.056   NaN  0.0033  0.0120  0.0151   
1       2440511 2025-04-04  132.0    NaN  0.035   NaN  0.0467  0.0271  0.0737   
2       2440512 2025-04-04   38.0  31.00  0.063  0.44  0.0028  0.0083  0.0111   
3       2440513 2025-04-04   50.0    NaN  0.056  1.31  0.0045  0.0168  0.0215   
4       2440514 2025-04-04   53.0  16.00  0.061  0.52     NaN  0.0032  0.0056   

      SO2 Sensor_id  TEMPERATURA  LLUVIA  PRESIONATM  HUMEDAD     RS  \
0  0.0047      ANL1        37.92     0.0       712.9     17.0  0.138   
1  0.0046     ANL10        38.37     0.0       701.2     10.0  0.198   
2  0.0106     ANL11         0.00     0.0       701.9     14.0  0.186   
3  0.0059     ANL12  

In [18]:
# Verificar nulos
print("\nNulos en sensores:")
print(sensores.isnull().sum())

print("\nNulos en registros:")
print(registros.isnull().sum())


Nulos en sensores:
Sensor_id          0
Zona               0
Calle              0
Municipio          0
CP                 0
Latitud            0
Longitud           0
Sistema            0
Bot_id             0
USE_IN_BOTS        0
website_visible    0
dtype: int64

Nulos en registros:
Registros_id         0
Dia                  0
PM10             72282
PM25            285840
O3              324296
CO              260565
NO1             665002
NO2             380506
NOx             660142
SO2             295805
Sensor_id            0
TEMPERATURA     594540
LLUVIA          642805
PRESIONATM      642805
HUMEDAD         594540
RS              642805
VIENTOVEL       642805
dtype: int64


In [19]:
# Fechas de inicio y fin
print("\nFechas:")
print(registros["Dia"].min(), "→", registros["Dia"].max())

# Duplicados
print("\nRegistros duplicados →", registros.duplicated().sum())

# Datos por sensor
conteo_por_sensor = registros["Sensor_id"].value_counts()
print("\nTop 5 sensores con más registros:")
print(conteo_por_sensor.head())

print("\nTop 5 sensores con menos registros:")
print(conteo_por_sensor.tail())

conteo_por_dia = registros["Dia"].value_counts().sort_index()
print("\nRegistros promedio por día:", round(conteo_por_dia.mean(), 2))


Fechas:
2018-01-01 08:00:00 → 2025-04-04 13:00:00

Registros duplicados → 0

Top 5 sensores con más registros:
Sensor_id
ANL10    56866
ANL9     56863
ANL11    56858
ANL8     56856
ANL12    56853
Name: count, dtype: int64

Top 5 sensores con menos registros:
Sensor_id
S10    12934
S12    12922
S8     12908
S11    12868
S15     4688
Name: count, dtype: int64

Registros promedio por día: 15.69


Cobertura diaria:
Si cada sensor puede registrar hasta 24 veces al día, el promedio de ~15 por día nos indica que hay pérdidas considerables.

In [20]:
# Sensores únicos en los registros
print("Sensores únicos en registros:", registros["Sensor_id"].nunique())
print("Sensores totales definidos:", sensores["Sensor_id"].nunique())

ids_registros = set(registros["Sensor_id"].unique())
ids_sensores = set(sensores["Sensor_id"].unique())

ids_comunes = ids_registros & ids_sensores

# IDs solo en registros (faltan metadatos)
ids_solo_registros = ids_registros - ids_sensores

# IDs solo en sensores (nunca registraron datos)
ids_solo_sensores = ids_sensores - ids_registros

print("IDs en ambos archivos:", sorted(ids_comunes))
print("IDs solo en registros:", sorted(ids_solo_registros))
print("IDs solo en sensores:", sorted(ids_solo_sensores))


Sensores únicos en registros: 29
Sensores totales definidos: 15
IDs en ambos archivos: ['ANL1', 'ANL10', 'ANL11', 'ANL12', 'ANL13', 'ANL15', 'ANL16', 'ANL2', 'ANL3', 'ANL4', 'ANL5', 'ANL6', 'ANL7', 'ANL8', 'ANL9']
IDs solo en registros: ['S1', 'S10', 'S11', 'S12', 'S13', 'S15', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
IDs solo en sensores: []


Al comparar los sensores definidos en los metadatos con los que aparecen en los registros, resalta hay 14 sensores que no tienen todos los campos de información (como ubicación o municipio). Esto limita su uso para análisis geográficos, por lo que se decide considerar solo con los sensores que están presentes en ambos archivos y tienen contexto completo.

In [21]:
# Generar un nuevo archivo con los registros filtrados
# que solo contengan los sensores que están en el archivo de sensores
registros_filtrados = registros[registros["Sensor_id"].isin(ids_comunes)]
registros_filtrados.to_excel("registros_filtrados.xlsx", index=False)

In [22]:
registros_filtrados.loc[:, "Sensor_id"] = registros_filtrados["Sensor_id"].str.strip()
sensores["Sensor_id"] = sensores["Sensor_id"].str.strip()

registros_con_zona = registros_filtrados.merge(
    sensores[["Sensor_id", "Zona"]],
    on="Sensor_id",
    how="left"
)

resumen_anl = registros_con_zona.groupby(["Sensor_id", "Zona"]).agg({
    "Registros_id": "count",
    "PM25": lambda x: x.isnull().mean(),
    "TEMPERATURA": lambda x: x.isnull().mean(),
    "VIENTOVEL": lambda x: x.isnull().mean()
}).rename(columns={
    "Registros_id": "Total_registros",
    "PM25": "% nulos PM25",
    "TEMPERATURA": "% nulos TEMPERATURA",
    "VIENTOVEL": "% nulos VIENTO"
})


dias_totales_anl = (registros_filtrados["Dia"].max() - registros_filtrados["Dia"].min()).days
registros_esperados_anl = dias_totales_anl * 24

# Cobertura estimada
resumen_anl["% cobertura estimada"] = resumen_anl["Total_registros"] / registros_esperados_anl

# Escalar a porcentajes legibles
resumen_anl = resumen_anl * 100
resumen_anl = resumen_anl.round(2)

display(resumen_anl)


,,Total_registros,% nulos PM25,% nulos TEMPERATURA,% nulos VIENTO,% cobertura estimada
Sensor_id,Zona,,,,,
ANL1,Guadalupe,5683900,22.30,56.02,62.08,89.37
ANL10,Apodaca,5686600,27.06,56.02,62.08,89.41
ANL11,Monterrey,5685800,36.32,56.01,62.07,89.40
ANL12,Obispado,5685300,25.86,56.07,62.14,89.39
ANL13,Juarez,5685300,25.69,56.01,62.07,89.39
ANL15,Pesqueria,3274300,98.44,23.61,34.14,51.48
ANL16,San Juan,1492300,95.92,1.09,1.09,23.46
ANL2,San Nicolás,5683500,18.25,55.99,62.07,89.36
ANL3,Santa Catarina,5682000,22.14,55.98,62.05,89.34


Tras filtrar los sensores con metadatos completos, se generó un resumen por sensor en el que se evaluó la cobertura de datos, así como el porcentaje de valores nulos en las variables clave: PM2.5, temperatura y velocidad del viento. 

La mayoría de los sensores ANL presentan una excelente cobertura estimada cercana al 89%, lo cual los hace adecuados para análisis temporal y modelado. 

Sin embargo, destacan dos casos: ANL15 (Pesquería) y ANL16 (San Juan), que presentan una cobertura baja (51.48% y 23.46%, respectivamente) y además una alta proporción de valores faltantes en PM2.5 (>95%). Estos sensores podrían considerarse poco confiables y candidatos para ser excluidos del análisis principal.

## Representatividad Geográfica de las Mediciones

En esta sección se muestra un mapa con la ubicación de los sensores ANL utilizados en el análisis.  
Esto permite evaluar la cobertura espacial de las mediciones en el área metropolitana de Monterrey  
y visualizar qué zonas están mejor monitoreadas.

In [ ]:
sensores_anl = sensores[sensores["Sensor_id"].str.startswith("ANL")]
mapa_sensores = folium.Map(location=[25.67, -100.31], zoom_start=11, min_zoom=8, max_zoom=15, tiles="CartoDB positron")

for _, row in sensores_anl.iterrows():
    folium.Marker(
        location=[row["Latitud"], row["Longitud"]],
        popup=f"{row['Sensor_id']} – {row['Zona']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mapa_sensores)

mapa_sensores